# Introduction à l'apprentissage automatique - Jour 2 - Classification

Dans ce notebook, vous allez apprendre à étudier un problème de classification
    
Ce notebook a été créé par [Chloé-Agathe Azencott](http://cazencott.info).

Ce noteboook utilise les librairies suivantes :
* python 3.4.3
* numpy 1.15.0
* matplotlib 2.2.2
* scikit-learn 0.19.2

Pour vérifier quelles versions de ces librairies vous utilisez, faites tourner la cellue ci-dessous en cliquant dessus puis en cliquant sur le bouton "Play" dans le menu au-dessus de cette fenêtre, ou en tapant Shift+Enter.

In [ ]:
import sys
print(sys.version)

import numpy
print(numpy.__version__)

import matplotlib
print(matplotlib.__version__)

import sklearn
print(sklearn.__version__)

# 1. Le problème du jour

Notre but aujourd'hui est d'utiliser la description visuelle d'un champignon pour prédire s'il est comestible ou non. 

Les données sont disponibles dans `data/mushrooms.csv`. Elles sont issues du jeu de données https://archive.ics.uci.edu/ml/datasets/Mushroom mais légèrement modifiées.

Vous pouvez commencer par l'ouvrir avec un éditeur de texte, ou un logiciel tableur, ou en ligne de commande avec la commande less. 

Il contient une première ligne (header) décrivant les colonnes, puis une ligne par champignon. Les valeurs des différentes variables sont toutes représentées par des lettres ; en voici la signification :
1. cap-shape: bell=b,conical=c,convex=x,flat=f, knobbed=k,sunken=s
2. cap-surface: fibrous=f,grooves=g,scaly=y,smooth=s
3. cap-color: brown=n,buff=b,cinnamon=c,gray=g,green=r, pink=p,purple=u,red=e,white=w,yellow=y
4. bruises?: bruises=t,no=f
5. odor: almond=a,anise=l,creosote=c,fishy=y,foul=f, musty=m,none=n,pungent=p,spicy=s
6. gill-attachment: attached=a,descending=d,free=f,notched=n
7. gill-spacing: close=c,crowded=w,distant=d
8. gill-size: broad=b,narrow=n
9. gill-color: black=k,brown=n,buff=b,chocolate=h,gray=g, green=r,orange=o,pink=p,purple=u,red=e, white=w,yellow=y
10. stalk-shape: enlarging=e,tapering=t
11. stalk-root: bulbous=b,club=c,cup=u,equal=e, rhizomorphs=z,rooted=r,missing=?
12. stalk-surface-above-ring: fibrous=f,scaly=y,silky=k,smooth=s
13. stalk-surface-below-ring: fibrous=f,scaly=y,silky=k,smooth=s
14. stalk-color-above-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o, pink=p,red=e,white=w,yellow=y
15. stalk-color-below-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o, pink=p,red=e,white=w,yellow=y
16. veil-type: partial=p,universal=u
17. veil-color: brown=n,orange=o,white=w,yellow=y
18. ring-number: none=n,one=o,two=t
19. ring-type: cobwebby=c,evanescent=e,flaring=f,large=l, none=n,pendant=p,sheathing=s,zone=z
20. spore-print-color: black=k,brown=n,buff=b,chocolate=h,green=r, orange=o,purple=u,white=w,yellow=y
21. population: abundant=a,clustered=c,numerous=n, scattered=s,several=v,solitary=y
22. habitat: grasses=g,leaves=l,meadows=m,paths=p, urban=u,waste=w,woods=d

La première colomne nous informe de la classe de chaque champignong, 'e' pour comestible (edible) et 'p' pour vénéneux (poisonous).

# 2. Mise en place du cadre de travail

Nous allons maintenant mettre en place notre cadre de travail. Plusieurs de ces étapes sont semblables à celles que nous avons suivies hier pour la régression et nous ne les détaillons pas ici.

## 2.1 Charger les librairies de science des données

In [ ]:
%pylab inline
import pandas as pd

## 2.2 Charger les données 

In [ ]:
df = pd.read_csv('data/mushrooms.csv')

In [ ]:
df.head()

# 2.3 Convertir les variables en valeurs numériques 

Nos variables sont pour l'instant catégorielles. Par exemple, pour la variable "forme du chapeau" (`cap shape`), `b` correspond à un chapeau campanulé (bell cap), `c` à un chapeau conique (conical cap), `f` à un chapeau plat (flat cap), `k` à un chapeau papillé (knobbed cap), `s` à un chapeau déprimé (sunken cap), and `x` à un chapeau convexe. 

Pour travailler avec ces données, il nous faut convertir ces catégories en valeurs numériques. Aujourd'hui, nous allons simplement convertir chaque lettre en un nombre entre 0 et le nombre total de catégorie, grâce à [preprocessing.LabelEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html).

Cet encodage n'est pas nécessairement idéal : un algorithme qui utilise la distance euclidienne va considérer qu'un chapeau convexe (`x` converti en 5) est plus proche d'un chapeau déprimé (`s` converti en 4) que d'un chapeau conique (`c` converti en 1), ce qui n'a pas beaucoup de sens. L'encodage [one-hot](http://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-categorical-features) pourrait être un meilleur choix. Cependant, il a l'inconvénient d'augmenter le nombre de variables, et de créer des variables corrélées. 

In [ ]:
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()

for col in df.columns:
    df[col] = labelencoder.fit_transform(df[col])

Nous pouvons de nouveau observer nos données :

In [ ]:
df.head()

## 2.4 Créer le cadre de sélection et d'évaluation de modèle

### Extraire X et y du dataframe df

In [ ]:
X = np.array(df.iloc[:, 1:]) # exclude first column
y = np.array(df.iloc[:, 0])  # first column only

In [ ]:
print(X.shape, y.shape)

__Question 1 :__ Combien d'échantillons (examples) notre jeu de données contient-il ? Combien de variables ?

__Réponse :__ _Écrire ici votre réponse._

### Séparer les données en un jeu d'entraînement et un de test

In [ ]:
from sklearn import model_selection

X_train, X_test, y_train, y_test = \
    model_selection.train_test_split(X, y,
                                    test_size=0.3 # 30% des données dans le jeu de test
                                    )

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

__Question 2 :__ Combien d'échantillons (examples) le jeu d'entraînement contient-il ?

__Réponse :__ _Écrire ici votre réponse._

### Mettre en place une validation croisée

In [ ]:
from sklearn import model_selection

# Créer un objet KFold qui permettra de cross-valider en 5 folds
kf = model_selection.KFold(n_splits=5,  # 5 folds
                           shuffle=True # mélanger les échantillons avant de créer les folds
                          )

# Utiliser kf pour partager le jeu d'entraînement en 5 folds. 
# kf.split retourne un iterateur (consommé après une boucle).
# Pour pouvoir se servir plusieurs fois des mêmes folds, nous transformons cet itérateur en liste d'indices :
kf_indices = list(kf.split(X_train))

### Choix du critère de sélection de modèle 

De nombreux critères de sélection de modèle sont à notre disposition pour les problèmes de classification. (Voir http://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter.) Aujourd'hui, nous allons utiliser le [score F1](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score).

__Question 3 :__ Pourquoi choisir le score F1 ? Quelle autre critère pourrait être approprié ?

__Réponse :__ _Écrire ici votre réponse._

# 3. Régression logistique 

Nous allons utiliser [l'implementation scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression) de la régression logistique, qui contient un paramètre de régularisation C.

## 3.1 Régression logistique non régularisée

__Question 3 :__ Pour utiliser une régression logistique très peu régularisée, faut-il choisir une grande valeur ou une faible valeur de C ? 

__Réponse :__ _Écrire ici votre réponse._

Nous allons maintenant calculer le score F1 moyen, en validation croisée sur le jeu d'entraînement, d'une régression logistique très peu régularisée.

In [ ]:
from sklearn import linear_model

# Créer un modèle de régression logistique 
model_logreg = linear_model.LogisticRegression(C=1e10)

# L'évaluer en validation croisée
f1_logreg = model_selection.cross_val_score(model_logreg, # modèle à entraîner
                                            X_train, y=y_train, # jeu d'entrainement
                                            scoring='f1', # score à utiliser
                                            cv=kf_indices # validation croisée à utiliser
                                            )

# Afficher le F1 moyen
print("F1 de la régression logistique : %.3f +/- %.3f" % (np.mean(f1_logreg), # moyenne
                                                         np.std(f1_logreg) # écart-type
                                                        ))

__Question 4 :__ Ce score vous parait-il bon ? Mauvais ?

__Réponse :__ _Écrire ici votre réponse._

Nous pouvons aussi visualiser les coefficients affectés à chacune des variables dans la régression logistique. Pour cela, nous commençons par ré-entrainer une régression logistique sur l'intégralité du jeu d'entraînement :

In [ ]:
# Entrainer une régression logistique sur l'ensemble du jeu d'entrainement
model_logreg.fit(X_train, y_train)

fig = plt.figure(figsize(12, 6))

num_features = X_train.shape[1]
plt.scatter(range(num_features), model_logreg.coef_)

plt.xlabel('Variables', fontsize=14)
feature_names = list(df.columns[1:])
tmp = plt.xticks(range(num_features), feature_names, 
                 rotation=90, fontsize=14)
tmp = plt.ylabel('Poids', fontsize=14)

tmp = plt.title('Coefficients de la regression logistique', fontsize=16)

Pour mieux comparer ces poids, nous pouvons aussi afficher leurs valeurs absolues.

In [ ]:
fig = plt.figure(figsize(12, 6))

num_features = X_train.shape[1]

plt.scatter(range(num_features), np.abs(model_logreg.coef_))

plt.xlabel('Variables', fontsize=14)
feature_names = list(df.columns[1:])
tmp = plt.xticks(range(num_features), feature_names, 
                 rotation=90, fontsize=14)
tmp = plt.ylabel('Poids', fontsize=14)

tmp = plt.title('Coefficients (valeur absolue) de la regression logistique', fontsize=16)

__Question 5 :__ Quelles variables vous paraissent les plus utiles pour prédire si un champignon est comestible ?

__Réponse :__ _Écrire ici votre réponse._

## 3.2 Régression logistique régularisée l1

Certaines des variables ont un poids très proche de 0 dans notre régression logistique. Une régularisation l1 permettrait de ramener les poids correspondant à 0, et d'éliminer ces variables.

Nous allons reprendre exactement le code utilisé hier pour le lasso, pour observer l'évolution du modèle de classification appris par une régression logistique régularisée l1 en fonction du coefficient de régularisation. Attention, nous devons remplacer `alpha` par `C`, et le critère d'évaluation (MSE) par le score F1.


Nous allons aussi utiliser la [commande magique time](https://ipython.readthedocs.io/en/stable/interactive/magics.html) pour mesurer le temps de calcul d'une cellule de notre notebook.

In [ ]:
%%time 
# Commande magique pour chronométrer le temps d'exécution

# Définir une grille de valeurs pour le coefficient de régularisation :
cvalues = np.logspace(-3, 1, 10)

f1_per_cval_lasso = [] # pour enregistrer les valeurs de F1 pour chacune des 20 valeurs de C
weights_per_cval_lasso = [] # pour enregistrer les coefficients associés à chaque variable,  
                       # pour les 10 valeurs de C
for cval in cvalues:
    # Créer un modèle de régression ridge
    model_lasso = linear_model.LogisticRegression(penalty='l1', C=cval)
    
    # Calculer la performance en validation croisée du modèle
    f1 = model_selection.cross_val_score(model_lasso, 
                                           X_train, y=y_train, 
                                           scoring='f1', 
                                           cv=kf_indices)
    f1_per_cval_lasso.append(f1)
    
    # Entrainer le modèle sur le jeu d'entrainement total 
    model_lasso.fit(X_train, y_train)
    
    # Enregistrer les coefficients de régression 
    weights_per_cval_lasso.append(model_lasso.coef_)

# Convertir weights_per_cval_lasso en un array aux bonnes dimensions
weights_per_cval_lasso = np.array(weights_per_cval_lasso)
weights_per_cval_lasso = np.reshape(weights_per_cval_lasso, (weights_per_cval_lasso.shape[0], 
                                                             weights_per_cval_lasso.shape[-1]))
    
plt.errorbar(cvalues, # abcisse
             np.mean(np.array(f1_per_cval_lasso), axis=1), # ordonnée : MSE moyenne
             yerr=np.std(np.array(f1_per_cval_lasso), axis=1)/np.sqrt(5) # barre d'erreur verticale
            )
plt.xscale('log') # utiliser une échelle logarithmique en abcisse

# Étiqueter les axes
tmp = plt.xlabel('Valeur de C', fontsize=14)
tmp = plt.ylabel('F1 moyen', fontsize=14)

# Titre
tmp = plt.title("Regression logistique regularisee l1", fontsize=14)

# Créer une figure
fig = plt.figure(figsize=(8, 5))

lines = plt.plot(cvalues, # abcisse = valeurs de alpha
                 weights_per_cval_lasso # ordonnée = valeurs des coefficients de régression
                ) 
plt.xscale('log') # échelle logarithmique en abcisse

# Afficher la légende
tmp = plt.legend(lines, # récupérer l'identifiant 
                 list(df.columns[1:]), # nom de chaque variable
                 frameon=False, # pas de cadre autour de la légende
                 loc=(1, 0),  # placer la légende à droite de l'image
                 fontsize=14)

tmp = plt.xlabel('Valeur de C', fontsize=14)
tmp = plt.ylabel('Coefficient', fontsize=14)

tmp = plt.title('Regression logistique regularisee l1', fontsize=16)

__Question 6 :__ La régularisation l1 permet-elle d'améliorer la performance de la régression logistique ? Cela vous surprend-il ?

__Réponse :__ _Écrire ici votre réponse._

__Question 7 :__ Quelles vous paraissent être les variables les plus importantes pour prédire si un champignon est comestible ?

__Réponse :__ _Écrire ici votre réponse._

Plutôt que de manipuler nous mêmes la validation croisée pour toutes les valeurs d'hyperparamètre, si nous cherchons uniquement à sélectionner le meilleur modèle (et non pas à examiner chacun des modèles comme nous l'avons fait jusqu'à présent), il est possible d'utiliser la fonction [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) du module `model_selection` de scikit-learn.

In [ ]:
from sklearn import model_selection

In [ ]:
%%time 
# Définir une grille d'hyperparamètres
param_grid = {'C': cvalues}

# Definir un modèle de recherche sur grille pour une régression logistique pénalisée l1
model_ridge = model_selection.GridSearchCV(linear_model.LogisticRegression(penalty='l1'),
                                         param_grid, 
                                         scoring='f1', 
                                         cv=kf_indices)

# Entraîner le modèle
model_ridge.fit(X_train, y_train)

# Meilleure valeur d'hyperparamètre
print("C optimal :", model_ridge.best_params_)

# Performance du meilleur modèle
print("F1 moyenne : %.3f" % model_ridge.best_score_)

__Question 8 :__ Les résultats vous paraissent-ils compatible avec l'expérience précédente, ou nous avions affiché le score F1 de chacun des modèles ?

__Réponse :__ _Écrire ici votre réponse._

__Question 9 :__ Comparer le temps d'exécution de ce code à celui que nous avions écrit à la main.

__Réponse :__ _Écrire ici votre réponse._

# 4. Classification avec une SVM

## 4.1 SVM Linéaire

Nous pouvons appliquer exactement la même recette algorithmique pour l'étude d'une SVM linéaire que pour celle d'une régression logistique régularisée. En effet, nous avons dans les deux cas un unique hyperparamètre, le coefficient de régularisation.

La classification par SVM dans scikit-learn est implémentée par [svm.SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).

In [ ]:
from sklearn import svm

In [ ]:
%%time

# Définir une grille de valeurs pour le paramètre C
param_grid = {'C': [0.05, 0.1, 0.5, 1., 5.]}

# Definir un modèle de recherche sur grille pour une SVM
#model_svc_lin = model_selection.GridSearchCV(svm.SVC(kernel='linear'),
#                                         param_grid, 
#                                         scoring='f1', 
#                                         cv=kf_indices)

# Implémentation dans le primal (à privilégier quand p < n)
model_svc_lin = model_selection.GridSearchCV(svm.LinearSVC(dual=False),
                                             param_grid, 
                                             scoring='f1', 
                                             cv=kf_indices)

# Entraîner le modèle
model_svc_lin.fit(X_train, y_train)

# Meilleure valeur d'hyperparamètre
print("C optimal :", model_svc_lin.best_params_)

# Performance du meilleur modèle
print("F1 moyenne : %.3f" % model_svc_lin.best_score_)

__Question 10 :__ Comment la performance de la SVM linéaire se compare-t-elle à celle de la régression logistique ?

__Réponse :__ _Écrire ici votre réponse._

Une SVM linéaire apprend une combinaison linéaire des variables décrivant les données. Nous pouvons comparer ce modèle à celui de la régression logistique.

In [ ]:
# Le modèle optimal, entraîné sur l'ensemble des données d'entrainement,
# est accessible dans model_svc_lin.best_estimator_
fig = plt.figure(figsize(12, 6))

num_features = X_train.shape[1]

# Afficher les coefficients de la régression logistique
plt.scatter(range(num_features), model_logreg.coef_,
           label="Regression logistique")

# Afficher les coefficients de la SVM linéaire
plt.scatter(range(num_features), model_svc_lin.best_estimator_.coef_,
           label="SVM lineaire")

# Légende
tmp = plt.legend(fontsize=14)

# Axe des abcisses
plt.xlabel('Variables', fontsize=14)
feature_names = list(df.columns[1:])
tmp = plt.xticks(range(num_features), feature_names, 
                 rotation=90, fontsize=14)

# Axe des ordonnées
tmp = plt.ylabel('Poids', fontsize=14)

# Titre
tmp = plt.title('Coefficients des modeles lineaires', fontsize=16)

__Question  :__ Comment le modèle appris par la régression logistique et celui appris par la SVM linéaire se comparent-ils ?

__Réponse :__ _Écrire ici votre réponse._

## 4.2 SVM non linéaire

L'utilisation d'un __noyau__ nous permet de créer des modèles non-linéaires, peut-être plus appropriés pour nos données. 

Cependant, il nous faut maintenant choisir aussi le noyau et ses hyperparamètres. En l'absence d'hypothèse forte sur le modèle, nous allons utiliser un noyau RBF gaussien.

In [ ]:
%%time

# Définir une grille de valeurs pour le paramètre C de la SVM
# et le paramètre gamma du noyau gaussien
param_grid = {'C': [0.05, 0.1, 0.5, 1., 5.], 
             'gamma': [0.01, 0.1, 0.5]}

# Definir un modèle de recherche sur grille pour une SVM
model_svc_rbf = model_selection.GridSearchCV(svm.SVC(kernel='rbf'),
                                         param_grid, 
                                         scoring='f1', 
                                         cv=kf_indices)

# Entraîner le modèle
model_svc_rbf.fit(X_train, y_train)

# Meilleure valeur d'hyperparamètre
print("Parametres optimaux : ", model_svc_rbf.best_params_)

# Performance du meilleur modèle
print("F1 moyenne : %.3f" % model_svc_rbf.best_score_)

__Question 11 :__ Comment la performance de la SVM à noyau gaussien se compare-t-elle à celle de la SVM linéaire ?

__Réponse :__ _Écrire ici votre réponse._

__Question  12 :__ Quel(s) inconvénient(s) voyez-vous à l'utilisation d'un noyau gaussien plutôt que linéaire ?

__Réponse :__ _Écrire ici votre réponse._

# 5. Arbres et forêts

## 5.1 Arbre de décision

Nous allons maintenant utiliser un type très différent d'algorithme nous permettant d'apprendre un modèle linéaire : les arbres de décision, implémentés dans scikit-learn par [tree.DecisionTreeClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html).

In [ ]:
from sklearn import tree

__Question 13 :__ Entre une SVM non-linéaire et un arbre de décision, lequel pensez-vous conduire à (1) un modèle interprétable (2) un modèle performant ? Lequel pensez-vous prendre plus de temps à entraîner ?

__Réponse :__ _Écrire ici votre réponse._

Nous allons considérer comme hyperparamètre le nombre maximal de niveaux de l'arbre de décision, `max_depth`.

In [ ]:
%%time

# Définir une grille de valeurs pour la profondeur de l'arbre de décision 
param_grid = {'max_depth': [2, 5, 10, 20, 30]}

# Definir un modèle de recherche sur grille pour un arbre de décision
model_tree = model_selection.GridSearchCV(tree.DecisionTreeClassifier(),
                                        param_grid, 
                                         scoring='f1', 
                                         cv=kf_indices)

# Entraîner le modèle
model_tree.fit(X_train, y_train)

# Meilleure valeur d'hyperparamètre
print("Paramètres optimaux : ", model_tree.best_params_)

# Performance du meilleur modèle
print("F1 moyenne : %.3f" % model_tree.best_score_)

__Question 14 :__ Que pensez-vous de cette performance ? Êtes-vous surpris ?

__Réponse :__ _Écrire ici votre réponse._

## 5.2 Interprétation de l'arbre de décision 

### Visualisation

Le code suivant, issu de la [documentation de scikit-learn](http://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html), nous permet de visualiser l'arbre de décision que nous venons d'apprendre.

In [ ]:
n_nodes = model_tree.best_estimator_.tree_.node_count
children_left = model_tree.best_estimator_.tree_.children_left
children_right = model_tree.best_estimator_.tree_.children_right
feature = model_tree.best_estimator_.tree_.feature
threshold = model_tree.best_estimator_.tree_.threshold

node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
is_leaves = np.zeros(shape=n_nodes, dtype=bool)
stack = [(0, -1)]  # seed is the root node id and its parent depth
while len(stack) > 0:
    node_id, parent_depth = stack.pop()
    node_depth[node_id] = parent_depth + 1

    # If we have a test node
    if (children_left[node_id] != children_right[node_id]):
        stack.append((children_left[node_id], parent_depth + 1))
        stack.append((children_right[node_id], parent_depth + 1))
    else:
        is_leaves[node_id] = True

print("L'arbre de décision a %s noeuds et "
      "la structure suivante :"
      % n_nodes)
for i in range(n_nodes):
    if is_leaves[i]:
        print("%snode=%s feuille." % (node_depth[i] * "\t", i))
    else:
        print("%snode=%s noeud test: aller au noeud %s si %s <= %s sinon "
              "aller au noeud %s."
              % (node_depth[i] * "\t",
                 i,
                 children_left[i],
                 df.columns[1+feature[i]],
                 threshold[i],
                 children_right[i],
                 ))
print()

__Question 15 :__ Cet arbre de décision vous parait-il si interprétable ?

__Réponse :__ _Écrire ici votre réponse._

### Importance des variables

Pour interpréter l'arbre de décision, nous pouvons aussi regarder l'importance de chaque variable. Elle est d'autant plus grande que la variable permet de réduire l'erreur de classification de l'arbre.

Nous remplaçons dans le code précédent `model_logreg.coefs_` (les coefficients du modèle linéaire) par `model_tree.best_estimator_.feature_importances_` (les importances de l'arbre de décision).

In [ ]:
fig = plt.figure(figsize(12, 6))

num_features = X_train.shape[1]

# Afficher les valeurs absolues des 
# coefficients de la régression logistique
plt.scatter(range(num_features), np.abs(model_logreg.coef_),
           label="Regression logistique")

# Afficher les importances de l'arbre de décision,
# multipliées par 10 pour être présentées sur la même échelle
plt.scatter(range(num_features), 10*model_tree.best_estimator_.feature_importances_,
           label="Arbre de decision")

# Légende
tmp = plt.legend(fontsize=14)

# Axe des abcisses
plt.xlabel('Variables', fontsize=14)
feature_names = list(df.columns[1:])
tmp = plt.xticks(range(num_features), feature_names, 
                 rotation=90, fontsize=14)

# Axe des ordonnées
tmp = plt.ylabel('Importance/Poids', fontsize=14)

# Titre
tmp = plt.title('Importance des variables', fontsize=16)

__Question 16 :__ Quelles sont maintenant les variables les plus importantes ? Comment cela se compare-t-il aux modèles précédents ?

__Réponse :__ _Écrire ici votre réponse._

## 5.3 Forêt aléatoire

Peut-on améliorer les performances de l'arbre de décision en utilisant une méthode ensembliste ? Nous allons utiliser ici une forêt aléatoire,  implémentée dans scikit-learn comme [ensemble.RandomForestClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html).

In [ ]:
from sklearn import ensemble

Nous avons maintenant deux hyperparamètres, la profondeur maximale de chaque arbre, et le nombre d'arbres dans la forêt. 

Nous modifions encore une fois le code permettant de choisir ces hyperparamètres, pour considérer le nombre d'arbres `n_estimators` comme hyperparamètre, et `ensemble.RandomForestClassifier` comme classifieur :

In [ ]:
%%time

# Définir une grille de valeurs pour les paramètres n_estimators
# et max_depth
param_grid = {'n_estimators': [10, 20, 50, 100, 300, 500],
             'max_depth': [2, 5, 10, 20]}

# Definir un modèle de recherche sur grille pour une forêt aléatoire
model_forest = model_selection.GridSearchCV(ensemble.RandomForestClassifier(),
                                             param_grid, 
                                             scoring='f1', 
                                             cv=kf_indices)

# Entraîner le modèle
model_forest.fit(X_train, y_train)

# Meilleure valeur d'hyperparamètre
print("Nombre d'arbres optimal :", model_forest.best_params_)

# Performance du meilleur modèle
print("F1 moyenne : %.3f" % model_forest.best_score_)

__Question 17 :__ Comment la performance de la forêt aléatoire se compare-t-elle aux performances précédentes ?

__Réponse :__ _Écrire ici votre réponse._

Nous pouvons encore une fois regarder l'importance de chaque variable, pour le meilleur modèle de forêt aléatoire, `model_forest.best_estimator_` :

In [ ]:
fig = plt.figure(figsize(12, 6))

num_features = X_train.shape[1]

# Afficher les valeurs absolues des 
# coefficients de la régression logistique
plt.scatter(range(num_features), np.abs(model_logreg.coef_),
           label="Regression logistique")

# Afficher les importances de l'arbre de décision,
# multipliées par 10 pour être présentées sur la même échelle
plt.scatter(range(num_features), 10*model_tree.best_estimator_.feature_importances_,
           label="Arbre de decision")

# Afficher les importances de la forêt aléatoire
plt.scatter(range(num_features), 10*model_forest.best_estimator_.feature_importances_,
           label="Foret aleatoire")

# Légende
tmp = plt.legend(fontsize=14)


# Axe des abcisses
plt.xlabel('Variables', fontsize=14)
feature_names = list(df.columns[1:])
tmp = plt.xticks(range(num_features), feature_names, 
                 rotation=90, fontsize=14)

# Axe des ordonnées
tmp = plt.ylabel('Importance', fontsize=14)

# Titre
tmp = plt.title('Importance des variables (foret aleatoire)', fontsize=16)

__Question 18 :__ Quelles sont maintenant les variables les plus importantes ? Comment cela se compare-t-il aux modèles précédents ?

__Réponse :__ _Écrire ici votre réponse._

# 6. Modèle final

__Question 19 :__ Lequel de ces modèles choisissez vous comme le plus performant pour classifier les champignons du jeu de test ? 

__Réponse :__ _Écrire ici votre réponse._

Vous allez maintenant évaluer le modèle que vous avez choisi sur le jeu de test :

In [ ]:
my_model = model_forest.best_estimator_ # TODO : insérez ici le nom du modèle que vous avez choisi.

# Prédire sur le jeu de test
y_pred = my_model.predict(X_test)

In [ ]:
# Pour évaluer la performance, nous allons avoir besoin de metrics
from sklearn import metrics

In [ ]:
# Évaluer la performance : score F1 sur le jeu de test
print("Score F1 en test du modele choisi : %.3f\n" % metrics.f1_score(y_test, y_pred))

# Évaluer la performance : matrice de confusion 
print("Matrice de confusion du modele choisi :\n",  metrics.confusion_matrix(y_test, y_pred))


__Question 20 :__ D'après la [documentation de metrics.confusion_matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html#sklearn.metrics.confusion_matrix), la matrice de confusion contient les nombres suivants :
``` 
    [[TN, FP]
    [FN, TP]]
```
Combien de champignons avez-vous classifiés comme comestible alors qu'ils ne le sont pas ?

__Réponse :__ _Écrire ici votre réponse._